In [22]:
import numpy as np
import random
import itertools

In [35]:
class HMM:
    def __init__(self,no_of_state,no_of_value,states,values,observered):
        self.emssion_matrix=np.array([ran/ran.sum() for ran in np.array([np.random.rand(no_of_value) for i in range(no_of_state)])]) #b[i][O]  --> probability to emit values[O] from states[i]
        
        self.initial_matrix= np.random.rand(no_of_state) #π[i] --> probability to start at states[i]
        self.initial_matrix/= self.initial_matrix.sum()
        
        self.transition_matrix=np.array([ran/ran.sum() for ran in np.array([np.random.rand(no_of_state) for i in range(no_of_state)])])  #a[i][j] --> probability from states[i] transit to states[j]
        self.states=states
        self.values=values
        self.observered=observered#list of values's index
        
        self.prev_initial_matrix=self.initial_matrix
        self.prev_emssion_matrix=self.emssion_matrix
        self.prev_transition_matrix=self.transition_matrix
        
        
        
    def debug(self):
        print('initial_matrix\n',self.initial_matrix)
        print('transition_matrix\n',self.transition_matrix)
        print('emssion_matrix\n',self.emssion_matrix)
        
    def forward(self,t,j,mode='eval',obs=None):
        if obs is None:
            obs=self.observered
        if t==0:
            return self.initial_matrix[j]*self.emssion_matrix[j][obs[t]]
        else:
            if mode=='eval':
                return sum([self.forward(t-1,i)*self.transition_matrix[i][j] for i in range(len(self.states))]) * self.emssion_matrix[j][obs[t]]
            elif mode=='decode':
                return max([self.forward(t-1,i)*self.transition_matrix[i][j] for i in range(len(self.states))]) * self.emssion_matrix[j][obs[t]]
            
    def backward(self,t,i,mode='eval',obs=None):
        if obs is None:
            obs=self.observered
        if t==len(obs)-1:
            return 1
        else:
            if mode=='eval':
                return sum([self.transition_matrix[i][j]*self.emssion_matrix[j][obs[t+1]]*self.backward(t+1,j) for j in range(len(self.states))])
            elif mode=='decode':
                return max([self.transition_matrix[i][j]*self.emssion_matrix[j][obs[t+1]]*self.backward(t+1,j) for j in range(len(self.states))])
            
    def train(self,obs=None,epochs=2):
        if obs is None:
            obs=self.observered
        def probit_at_i(t,i,ob):#Gamma
            numerator=self.forward(t,i)*self.backward(t,i)#sum probability of all path passing through state[i] at time t
            denominator=sum([self.forward(t,j)*self.backward(t,j) for j in range(len(self.states))])
            return numerator/denominator
        def probit_transit_i_j(t,i,j,ob):#epsilon
            numerator=self.forward(t,i)*self.transition_matrix[i][j]*self.emssion_matrix[j][ob[t+1]]*self.backward(t+1,j)#sum probability of all path transit from state[i] to state[j] at time t
            denominator=sum([sum([self.forward(t,m)*self.transition_matrix[m][n]*self.emssion_matrix[n][ob[t+1]]*self.backward(t+1,n) for n in range(len(self.states))]) for m in range(len(self.states))])
            return numerator/denominator
        
        for epoch in range(epochs):
            for ob in obs:
                for i in range(len(self.states)):
                    self.initial_matrix[i]=probit_at_i(0,i,ob)
                for i, j in itertools.product(range(len(self.states)),range(len(self.states))):
                    self.transition_matrix[i][j]=sum([probit_transit_i_j(t,i,j,ob) for t in range(len(ob)-1)])/sum([probit_at_i(t,i,ob) for t in range(len(ob)-1)])
                for j, k in itertools.product(range(len(self.states)),range(len(self.values))):   
                    total=0
                    for t in range(len(ob)):
                        if ob[t]==k:
                            total+=probit_at_i(t,i,ob)
                    self.emssion_matrix[j][k]=total/sum([probit_at_i(t,i,ob) for t in range(len(ob))])

In [126]:
test=HMM(2,3,["Rainy", "Sunny"],["walk", "shop", "clean"],[2, 0, 1, 1, 2, 0])

In [127]:
test.train([[2, 0, 1, 1, 2, 0],[0, 0, 1, 1, 2, 0],[2, 1, 2, 1, 2, 0]],250)

In [128]:
test.debug()

initial_matrix
 [0. 1.]
transition_matrix
 [[0.         0.99803068]
 [0.7873343  0.21094684]]
emssion_matrix
 [[0.1149929  0.31986729 0.47922235]
 [0.21684415 0.29336996 0.51488945]]
